In [ ]:
# import the positions

import pickle

# Loading the lists from the file for training
with open('./PositionsAndGts_train.pkl', 'rb') as file:
    startingPositions_train = pickle.load(file)
    gts_train = pickle.load(file)


# Loading the lists from the file for validation
with open('./PositionsAndGts_valid.pkl', 'rb') as file:
    startingPositions_valid = pickle.load(file)
    gts_valid = pickle.load(file)

In [ ]:
## shuffling two lists together
import random
import numpy as np

StartingPositions = []
Gts =[]

StartingPositions_valids = []
Gts_valids =[]


def shuffle_and_convert(starting_positions, gts):
    # Create a combined list of tuples
    combined_lists = list(zip(starting_positions, gts))

    # Shuffle the combined list
    random.shuffle(combined_lists)

    # Unzip the shuffled list
    shuffled_list1, shuffled_list2 = zip(*combined_lists)

    # Convert the tuples back to lists and then to numpy arrays
    starting_positions_shuffled = np.array(list(shuffled_list1))
    gts_shuffled = np.array(list(shuffled_list2))

    return starting_positions_shuffled, gts_shuffled

# Assuming startingPositions_train, gts_train, startingPositions_valid, gts_valid are defined
StartingPositions_train, Gts_train = shuffle_and_convert(startingPositions_train, gts_train)
StartingPositions_valid, Gts_valid = shuffle_and_convert(startingPositions_valid, gts_valid)

# print(StartingPositions_train[0]) # Uncomment if checking is needed
# print(StartingPositions_valid[0]) # Uncomment if checking is needed



In [ ]:
from Generator import *

In [ ]:
# Check the load_patch function

import matplotlib.pyplot as plt

num = 5257
patch = load_patch(StartingPositions_train[num][3], StartingPositions_train[num][4], StartingPositions_train[num][0],3,256)

f, (ax1) = plt.subplots(1, 1, sharey=True)
ax1.imshow(patch)
print(Gts_train[num])
print(StartingPositions_train[num][0])


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Flatten, Dense, concatenate, BatchNormalization, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from accum_optimizers import AccumOptimizer  # Import the actual AccumOptimizer class for H2G-Net model
from tensorflow.keras.losses import CategoricalFocalCrossentropy
import tensorflow_addons as tfa

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU with index 0

previous_model = load_model('./pw_tumour_mobilenetv2_model.h5', compile=False)

# Define the input shapes
input_shape = (256, 256, 3)

# Freeze the layers of level 3 branch
for layer in previous_model.layers[0:-12]:
   layer.trainable = False

previous_model.layers.pop()

# Apply L2 regularization to specific layers
for layer in previous_model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.kernel_regularizer = tf.keras.regularizers.l2(0.04)


# Define the inputs for level 3 (Add level 5 branches if for multi-level netweork)
input_level3 = Input(shape=input_shape)

# Create the level 3 branch
level3_branch = previous_model(input_level3)

output = Dense(2, activation='softmax')(level3_branch)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Create the CNN model
model = Model(inputs=input_level3, outputs=output)
model.summary()

# Compile the model
model.compile(optimizer=optimizers.Adam(1e-4), 
              loss=CategoricalFocalCrossentropy(), 
              metrics=['acc', tfa.metrics.F1Score(num_classes=2, average="macro")])

# Callback for early stopping (to stop training when validation loss does not improve)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.1)

# Callback to save the best model based on validation loss
model_checkpoint = ModelCheckpoint(
    'best_model.h5',  # Path where to save the model
    monitor='val_loss',  # Metric name to monitor
    save_best_only=True,  # Only save the best model
    mode='min',  # The best model is the one which minimizes the loss
    verbose=1  # Log a message for each improvement
)

# Set the directory path where the data is stored
data_directory = 'path_to_data_directory'

# Create the custom data generator
train_generator = CustomDataGenerator(StartingPositions_train, Gts_train, 4)
valid_generator = CustomDataGenerator(StartingPositions_valid, Gts_valid, 4)

# Train the model with the callbacks
model.fit(train_generator, validation_data=valid_generator, batch_size=None, verbose=1, callbacks=[early_stopping, model_checkpoint])



In [ ]:
# Convert the best result to onnx

import tf2onnx
import onnx

# Load the best model saved during training
best_model_path = 'best_model.h5'
model = tf.keras.models.load_model(best_model_path, custom_objects={'F1Score': tfa.metrics.F1Score})  # Ensure to include custom_objects if you used any in your model

# Convert the loaded model to ONNX format
# Note: You might need to adjust the opset version based on the requirements of your model and compatibility with ONNX
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)

# Save the ONNX model to a file
onnx_file_path = "./Transfered1.onnx"
onnx.save(onnx_model, onnx_file_path)

print(f"ONNX model saved to {onnx_file_path}")
